In [1]:
import psycopg2
import numpy as np
import pandas as pd
import psycopg2.extras as extras
from sqlalchemy import create_engine

from pathlib import Path
import io

data_dir = Path.cwd().parent / 'data' / 'cleaned'
list(data_dir.glob('*.csv'))

[WindowsPath('w:/OU/OneDrive - University of Oklahoma/MSinDSA/Spring-2023/DSA-5900/data/cleaned/business.csv'),
 WindowsPath('w:/OU/OneDrive - University of Oklahoma/MSinDSA/Spring-2023/DSA-5900/data/cleaned/review.csv'),
 WindowsPath('w:/OU/OneDrive - University of Oklahoma/MSinDSA/Spring-2023/DSA-5900/data/cleaned/user.csv')]

In [2]:
db_config = {
    "database": "yelp_db",
    "user": "postgres",
    "password": "admin",
    "host": "localhost",
    "port": "5432"
}

engine = create_engine(
    f'postgresql+psycopg2://{db_config["user"]}:{db_config["password"]}@{db_config["host"]}:{db_config["port"]}/{db_config["database"]}')

## Inserting Business Data

In [10]:
business_df = pd.read_csv(data_dir / 'business.csv')

cats = [i for i in business_df.columns if i.startswith('cat_')]
attrs = [i for i in business_df.columns if i.startswith('attr_')]
main_cols = [i for i in business_df.columns if i not in cats + attrs]

# Divide cats into two groups to avoid exceeding the maximum number of parameters in a query by
# randomly selecting half of the columns
cats_1 = list(np.random.choice(cats, int(len(cats) / 2), replace=False))
cats_2 = [i for i in cats if i not in cats_1]

C:\Users\biswa\AppData\Local\Temp\ipykernel_8028\565852556.py:1: DtypeWarning: Columns (11,15,16,17,18,20,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  business_df = pd.read_csv(data_dir / 'business.csv')


In [11]:
attrs_df = business_df[['business_id'] + attrs].copy()
cats_1_df = business_df[['business_id'] + cats_1].copy()
cats_2_df = business_df[['business_id'] + cats_2].copy()
business_df = business_df[main_cols].copy()

In [12]:
business_df.to_sql("business", con=engine, if_exists='replace', index=False, chunksize=1000)
attrs_df.to_sql("business_attr", con=engine, if_exists='replace', index=False, chunksize=1000)
cats_1_df.to_sql("business_cat1", con=engine, if_exists='replace', index=False, chunksize=1000)
cats_2_df.to_sql("business_cat2", con=engine, if_exists='replace', index=False, chunksize=1000)

3003

In [14]:
# Clean up
# del business_df, attrs_df, cats_1_df, cats_1_df